In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from databroker import DataBroker as db
from sixtools.rixs_wrapper import make_scan, centroids_to_spectrum, get_rixs
from rixs.process2d import apply_curvature, optimize_curvature
from IPython.display import clear_output, display

%matplotlib widget

## Define and execute processing

In [ ]:
scan_id = 51402
process_dicts_high_2theta = {'light_ROI': [800, 1230, 175, 1500],
                                 'curvature': np.array([0., 0., 0.]),
                                 'bins': 0.5,
                                 'background': None}

table = pd.concat(t for event in db[scan_id].data('rixscam_centroids') for t in event)
light_ROI = process_dicts_high_2theta['light_ROI']
choose = np.logical_and.reduce((table['x_eta'] >= light_ROI[0],
                                    table['x_eta'] < light_ROI[1],
                                    table['y_eta'] >= light_ROI[2],
                                    table['y_eta'] < light_ROI[3],
                                    table['sum_regions'] >= 0.1,
                                    table['sum_regions'] < np.inf))
photon_events = table[choose][['x_eta', 'y_eta', 'sum_regions']].values

In [ ]:
curvature_guess = process_dicts_high_2theta['curvature']
bins = process_dicts_high_2theta['bins']
curvature = optimize_curvature(photon_events, curvature_guess, bins)

In [ ]:
fig_spectrum, ax_spectrum = plt.subplots(num=0, clear=True)

original_spectrum = apply_curvature(photon_events, curvature_guess, bins)
optimized_spectrum = apply_curvature(photon_events, curvature, bins)

ax_spectrum.plot(original_spectrum[:,0], original_spectrum[:,1], '.-', label='Original')
ax_spectrum.plot(optimized_spectrum[:,0], optimized_spectrum[:,1], '.-', label='Optimized')
ax_spectrum.set_xlabel('Pixels')
ax_spectrum.set_ylabel('Intensity')
ax_spectrum.legend()
ax_spectrum.set_xlim([1280, 1320])
#display(fig_spectrum)